In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [3]:
heart_df = pd.read_csv('../Datasets/heart.csv')

In [4]:
heart_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
heart_df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [6]:
heart_df.shape

(303, 14)

In [7]:
heart_df.target.value_counts()

target
1    165
0    138
Name: count, dtype: int64

1 --> Heart disease

0 --> Healthy Heart

In [8]:
X = heart_df.drop(columns='target',axis=1)

Y = heart_df.target

In [9]:
print(X)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  
0        0   0     1  
1        0   0     2  
2        2   0    

In [10]:
print(Y)

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64


Model Selection

In [14]:
models = [LogisticRegression(max_iter=1000),SVC(kernel='linear'),RandomForestClassifier(random_state=0),KNeighborsClassifier()]

In [15]:
def compare_models_cross_validation():
    for model in models:
        cv_score = cross_val_score(model,X,Y,cv=5)

        mean_acc = np.mean(cv_score)
        mean_acc = mean_acc*100
        mean_acc = round(mean_acc,2)

        print('Cross Validation accuracies for the',model,'=', cv_score)
        print('Acccuracy score of the ',model,'=',mean_acc,'%')
        print('---------------------------------------------------------------')

In [16]:
compare_models_cross_validation()

C:\Users\Jash\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Cross Validation accuracies for the LogisticRegression(max_iter=1000) = [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Acccuracy score of the  LogisticRegression(max_iter=1000) = 82.83 %
---------------------------------------------------------------
Cross Validation accuracies for the SVC(kernel='linear') = [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
Acccuracy score of the  SVC(kernel='linear') = 82.83 %
---------------------------------------------------------------
Cross Validation accuracies for the RandomForestClassifier(random_state=0) = [0.85245902 0.90163934 0.81967213 0.81666667 0.8       ]
Acccuracy score of the  RandomForestClassifier(random_state=0) = 83.81 %
---------------------------------------------------------------
Cross Validation accuracies for the KNeighborsClassifier() = [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Acccuracy score of the  KNeighborsClassifier() = 64.39 %
----------------------------------------------------------

Comparing the models with different hyperparameters using GridSearchCV

In [39]:
models_list = [LogisticRegression(max_iter=1000),SVC(),KNeighborsClassifier(),RandomForestClassifier(random_state=0)]

In [47]:
model_hyperparameters = {

    'log_reg_hyperparams':{

        'C':[1,5,10,20]
    },
    'svc_hyperparams':{
        'kernel':['linear','poly','sigmoid','rbf'],
        'C':[1,5,10,20]
    },
    'KNN_hyperparams':{
        'n_neighbors':[3,5,10]
    },
    'Random_forest_hyperparams':{
        'n_estimators':[10,20,50,100]
    }
}

In [48]:
type(model_hyperparameters)

dict

In [49]:
print(model_hyperparameters.keys())

dict_keys(['log_reg_hyperparams', 'svc_hyperparams', 'KNN_hyperparams', 'Random_forest_hyperparams'])


In [50]:
model_keys = list(model_hyperparameters)

print(model_keys)

['log_reg_hyperparams', 'svc_hyperparams', 'KNN_hyperparams', 'Random_forest_hyperparams']


In [51]:
model_hyperparameters[model_keys[2]]

{'n_neighbors': [3, 5, 10]}

Applying GridSearchCV

In [52]:
def ModelSelection(list_of_models,hyperparameters):

    result = []
    i = 0
    
    for model in list_of_models:
        key = model_keys[i]

        params = hyperparameters[key]
        i = i+1
        print(model)
        print(params)

        classifier = GridSearchCV(model,params,cv=5)

        classifier.fit(X,Y)

        result.append({
            'model used':model,
            'highest score':classifier.best_score_,
            'best hyperparameters':classifier.best_params_
        })
    result_df = pd.DataFrame(result,columns=['model used','highest score','best hyperparameters'])

    return result_df

In [53]:
ModelSelection(models_list,model_hyperparameters)

LogisticRegression(max_iter=1000)
{'C': [1, 5, 10, 20]}


C:\Users\Jash\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Jash\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

SVC()
{'kernel': ['linear', 'poly', 'sigmoid', 'rbf'], 'C': [1, 5, 10, 20]}
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}


,model used,highest score,best hyperparameters
0,LogisticRegression(max_iter=1000),0.831585,{'C': 5}
1,SVC(),0.828306,"{'C': 1, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.643880,{'n_neighbors': 5}
3,RandomForestClassifier(random_state=0),0.838087,{'n_estimators': 100}
